In [1]:
import numpy as np
from numpy import linalg as la
import queue

In [2]:
from sympy import symbols, Eq, solve
from sympy.matrices import Matrix, eye, zeros, ones, diag
from sympy import re,I,E, symbols

In [3]:
from sympy import init_printing
init_printing(use_latex='mathjax')
from IPython.display import display

In [4]:
k =np.array( [[5,-1,9],
     [0.5,20,6],
     [7,7,1]])

In [6]:
ka = la.inv(k)
print(ka)
ka = ka.T
print(ka)
ans = np.multiply(k,ka)

[[ 0.01594203 -0.04637681  0.13478261]
 [-0.03007246  0.04202899  0.01847826]
 [ 0.09891304  0.03043478 -0.07282609]]
[[ 0.01594203 -0.03007246  0.09891304]
 [-0.04637681  0.04202899  0.03043478]
 [ 0.13478261  0.01847826 -0.07282609]]


In [7]:
ans

array([[ 0.07971014,  0.03007246,  0.89021739],
       [-0.02318841,  0.84057971,  0.1826087 ],
       [ 0.94347826,  0.12934783, -0.07282609]])

In [ ]:
#representation of A wrt to basis vectors provided
def Represent(A,T,show_calc=True):
  assert la.matrix_rank(A) == A.shape[0], "rank deficant matrix A"
  
  assert T.shape == A.shape, 'issuficiant basis vectors'
  
  
  T_inv  = la.inv(T)
  
 
  TiA = np.dot(T_inv,A)
  
  A_rep = np.float16(np.dot(TiA,T))
  
  if show_calc:
    print("Given A to be represented = ")
    print(A)
    print(" T  := ")
    print(T)
    print("T inverse := ")
    print(T_inv)
    print(" T_inv . A = ")
    print(TiA)
    print(" representation_of_A = T_inv . A . T =  ")
    print(A_rep)
  return A_rep

def check_singular(matrix):
  assert la.matrix_rank(matrix) == matrix.shape[0], "rank deficient"
  print("Matrix is non singular")
  return
def diagonalize_from_CCF(matrix):     # if A in controller canonical form (CCF)
  print("A in CCF")
  check_singular(matrix)
  eig, _ = la.eig(matrix)
  print(f"eigenvalues  := {eig}")
  rank = la.matrix_rank(matrix)
  ev = []
  assert len(np.unique(eig))== rank, "ununique eigenvectors"
  print("Matrix has unique eigen values")
  print("calculation T from the eigen vectors and mapping [1 lmda**1 lmda**2 ...].T")
  for k,i in enumerate(eig):
    d = np.array([i**j for j in range(rank)])
    
    ev.append(d)
  ev = np.array(ev).T
  
  A_diag = Represent(matrix,ev,show_calc=True)
  return

class system():
  def __init__(self):
    self.ABCD_defined = False
    return
  def define_ABCD(self,A=None,B=None,C=None,D=None,show=True):
    
    self.A = A if A is not None else None
    self.B = B if B is not None else None
    self.C = C if C is not None else None
    self.D = D if D is not None else None
    self.ABCD_defined = True
    return
  def check_rank(self,matrx):
    rank = la.matrix_rank(matrx)
    print("rank :=",rank)
    return rank
  def detrm(self,matrix):
    detr = la.det(matrix)
    print("determinant is :=".format(detr))
    return detr
  
  def build_T_for_controllibilty(self,coef):
    # we need to provide the coef of det(sI-A) = s3 - a1s2 - a2s- a3 as (a3, a2, a1)
    print(" T matrix for controllibility representation")
    
    #assert len(coef)==len(self.A), "isufficient coef"
    M = []
    k =1
    
    mk = [i for i in coef[1:]]+[1]
    B = []
    for i in range(len(coef)):
      m = mk[i:]+[0 for i in range(i)]
      M.append(m)
      
      b = la.matrix_power(A,i) 
      b = b@self.B
      B.append(b.reshape(-1))
    
    M = np.array(M)
    C = np.array(B).T
    #assert la.matrix_rank(C) == C.shape[0], "O is rank deficient"
    T = C@M
    #assert la.matrix_rank(T) == t.shape[0] , "T is non invertible"
    print("M := ")
    print(M)
    
    print("C = [B AB A2B]")
    print(C)

    print("T = C.M :=")
    print(T)
    C = Matrix(C)
    return T,C

  def build_T_for_observibility(self,coef):
    # we need to provide the coef of det(sI-A) = s3 - a1s2 - a2s- a3 = 0  as (a3, a2, a1)
    print(" T matrix for observability representation")
    #assert len(coef)==len(self.A), "isufficient coef"
    M = []
    k =1
    
    mk = [i for i in coef[1:]]+[1]
    O = []
    for i in range(len(coef)):
      m = mk[i:]+[0 for i in range(i)]
      M.append(m)
      
      c = la.matrix_power(A,i) 
      
      c = self.C @ c
      
      O.append(c.reshape(-1).tolist())
    
    M = Matrix(np.array(M))
    O = Matrix(np.array(O))
    
    #assert la.matrix_rank(O) == O.shape[0], "O is rank deficient"
    
    T_inv = M*O
    T = T_inv.inv()
    print("M := ")
    display(M)
    
    print("O = [[C][CA][CA2]]")
    display(O)

    print("T = inv_(MO) :=")
    display(T)
    return T,O
  
  def ABC_similarity_transform(self,T,show_calc=True):
    ## similarity transformation doesnt change the eigen values of the system
    ## hence transfer function is preserved.
    #check_singular(T)
    self.T = T
    T_inv = la.inv(T)
    self.C_bar = np.dot(self.C,T)
    self.B_bar = np.dot(T_inv,self.B)
    self.A_bar = Represent(A,T,show_calc)

    print("A_bar = T_inv . A . T ")
    print(self.A_bar)
    print("B_bar = T_inv . B")
    print(self.B_bar)
    print("C_bar = C . T")
    print(self.C_bar)
    return self.A_bar,self.B_bar,self.C_bar




In [ ]:
def diagonalize_A(matrix):
  matrix = Matrix(matrix)
  assert matrix.det() !=0, 'Matrix rank deficient, det=0'
  eigenvects = []
  for tup in matrix.eigenvects():
    for j in tup[2]:
       
        eigenvects = eigenvects +list(j)
  T = Matrix(matrix.shape[0],matrix.shape[0],eigenvects).transpose()
  print("T matrix")
  display(T)
  T_inv = T.inv() 
  print("T_inv")
  display(T_inv)
  Ti_A = T_inv*matrix
  A_bar = Ti_A*T
  print('A_diagonal')
  display(A_bar)
  return A_bar
#T= diagonalize_A(A)

In [ ]:
A = np.array([[0,1],[-2,-3]])
T  = diagonalize_A(A)

T matrix


⎡-1/2  -1⎤
⎢        ⎥
⎣ 1    1 ⎦

T_inv


⎡2   2 ⎤
⎢      ⎥
⎣-2  -1⎦

A_diagonal


⎡-2  0 ⎤
⎢      ⎥
⎣0   -1⎦

In [ ]:
def coefs_of_si_A_detrm(matrix):
  # a4.x**3 + a3.x**2 + a2.x + a1 : out: [a1,a2,a3,a4]
  s = symbols('s')
  
  sI = eye(matrix.shape[0])*s
  
  si_A = sI - matrix
  det = si_A.det()
  cdict = det.as_coefficients_dict()
  print('Determinant of SI-A')
  display(det)
  coefs =[]
  for i in range(sI.rank()+1):
    try:
      c = cdict[s**i]
    except:
      c = 0
    coefs.append(c)
  return coefs

#coefs_of_si_A_detrm(A)

In [ ]:
def ABC_similarity_transform(A,B,C,T,show_calc=True):
  A = Matrix(A)
  B = Matrix(B)
  C = Matrix(C)
  display(C)
  T = Matrix(T)
  t_inv = T.inv()
  A_T = A*T
  A_bar = t_inv*A_T
  if C:
      C_bar = C*T
  if B:
    B_bar = t_inv*B

  if show_calc:
    print("T_inv :=")
    display(t_inv)
    print("Calculate A_bar = T_inv * A * T")
    print("A*T  :=")
    display(A_T)
    print('A_bar  :=')
    display(A_bar)
    if B:
      print("Calculation of B_bar = T_inv * B  :=")
      display(B_bar)
    if C:
      print("Calculate C_bar = C * T  :=")
      display(C_bar)
  return A_bar,B_bar,C_bar 




In [ ]:
class Controller():
  def __init__(self,A_ccf,B_ccf,C_ccf,T_ccf):
    self.A = Matrix(A_ccf)
    self.B = Matrix(B_ccf)
    self.C = Matrix(C_ccf)
    self.T = Matrix(T_ccf)
  def build_k_matrix(self,d):
    
    if d==2:
      k1,k2 = symbols('k1 k2')
      return Matrix([[k1,k2]])
    elif d==3 :
      k1,k2,k3 = symbols('k1 k2 k3')
      return Matrix([[k1,k2,k3]])
    elif d==4 :
      k1,k2,k3 ,k4= symbols('k1 k2 k3 k4')
      return Matrix([[k1,k2,k3,k4]])
    else:
      return None
  
  def pole_assg_Similarity_transform(self,eigenvalues,show_calc=True):
    #for control input
    d = self.A.shape[0]
    k = self.build_k_matrix(d)
    print("K matrix")
    display(k)
    
    t_inv = self.T.inv()
    A_BK = self.A - self.B*k
    s = symbols('s')
    sI = eye(d)*s
    matrix = sI - A_BK
    det = matrix.det()
    cdict = det.as_coefficients_dict()
    if show_calc:
      print("Solving using similarity transform \n")
      print("B  :=")
      display(self.B)
      print('BK :=')
      display(self.B*k)
      print('A-BK :=')
      display(A_BK)
      
      print('SI-(A-BK)')
      display(matrix)
    Alphas = coefs_of_si_A_detrm(diag(eigenvalues,unpack=True))
    Alphas.reverse()
    print("required coef of det of (si - lambda) , where lambda are the new pole location")
    print(Alphas)
    print("det(SI-(A-BK))")
    display(det)
    
    print('solve for k1,k2,k.... and enter')
    ks = []
    for i in range(d):
      j=input(f'enter K{i+1}  : ')
      ks.append(j)
    
    k_bar = Matrix([ks])
    print("K_bar entered")
    display(k_bar)
    K = k_bar*t_inv
    K = K.reshape(1,len(K))
    print("K = K_bar*T_inv  :=")
    display(K)
    return k_bar,K

  def pole_assg_Similarity_transform_2(self,eigenvalues,show_calc=True):
    #for observer input
    d = self.A.shape[0]
    L = self.build_k_matrix(d)
    L = L.transpose()
    print("L matrix")
    display(L)
    
    t_inv = self.T.inv()
    A_CL = self.A - L*self.C
    s = symbols('s')
    sI = eye(d)*s
    matrix = sI - A_CL
    det = matrix.det()
    cdict = det.as_coefficients_dict()
    if show_calc:
      print("Solving using similarity transform \n")
      print("C  :=")
      display(self.C)
      print('L*C :=')
      display(L*self.C)
      print('A-CL :=')
      display(A_CL)
      
      print('SI-(A-CL)')
      display(matrix)
    Alphas = coefs_of_si_A_detrm(diag(eigenvalues,unpack=True))
    Alphas.reverse()
    print("required coef of det of (si - lambda) , where lambda are the new pole location")
    print(Alphas)
    print("det(SI-(A-BK))")
    display(det)
    
    print('solve for l1,l2,l3.... and enter')
    ks = []
    for i in range(d):
      j=input(f'enter L{i+1}  : ')
      ks.append(j)
    
    k_bar = Matrix([ks])
    print("L_bar entered")
    display(k_bar)
    K = k_bar*t_inv
    K = K.reshape(1,len(K))
    print("K = K_bar*  :=")
    display(K.transpose())
    return k_bar,K
  
def tracking_gain_N(A,B,C,k,show_calc=True):
  print("\n Tracking problem, u = Nr - kx ")
  print("\n G_cl(s) = C * (sI - (A-BK))^-1 * B \n")
  print("G_cl(0) = C * (-(A-BK))^-1 * B ")
  print("N_bar = G_cl^-1;\n")
  A = Matrix(A)
  B = Matrix(B)
  C = Matrix(C)
  k = Matrix(k)

  s = symbols('s')
  d = A.rank()
  sI = eye(d)*s
  A_bk = A - B*k
  
  A_bk_inv = A_bk.inv()
  G_cl = - C*A_bk_inv*B
  if show_calc:
    print("(A-Bk)")
    display(A_bk)
    print("(A-Bk)^-1 :=")
    display(A_bk_inv)
    print("G_cl(0) = -C*(A-BK)^-1 *B := ")
    display(G_cl)
  N_bar = G_cl.inv()
  
  print("N_bar")
  display(N_bar)
  return

      
    

In [ ]:
def Ackkermans_formula(A,B,C,Control_matrix,eigenvalues=None,mode='controller',):
    print(f'\n Using Ackkermans formula for finding control input K/L for {mode} \n')
    A = Matrix(A)
    B = Matrix(B)
    C = Matrix(C)
    cont = Matrix(Control_matrix)
    Alphas = []
    if eigenvalues:
      Alphas = coefs_of_si_A_detrm(diag(eigenvalues,unpack=True))
      
    else:
      print("assuming ur putting a3,a2,a1 from the desired characteristic eq: a3*s2 + a2*s + a1")
      for i in range(A.shape[0]+1):
        c = int(input(f" input a{i+1} :"))
        Alphas.append(c)
    Alphas
    
    for i,j in enumerate(Alphas):
      if i==0:
        I = eye(A.shape[0])
        cv = j*I
        print(f"{j}*I :")
        display(cv)
        alpha_d = cv
        continue
      print(f"{j}*A**{i}")
      cv = j*A**i
      display(cv)
      
      alpha_d = alpha_d+ cv
    print('sum')
    display(alpha_d)
    if mode=='controller':
      print("controllibility MAtrix")
      display(cont)
      cont_inv = cont.inv()
      print("Cont_inv :=")
      display(cont_inv)
      print(" K = [0 ... 0 1]*C_inv*Alpha_d(A)  : = ")
      zerone= Matrix([[0 for i in range(A.shape[0]-1)]+[1]])
    
      K  = zerone * cont_inv * alpha_d
      display(K)
    elif mode=="observer":
      print("observibility matrix O")
      display(cont)
      O_inv = cont.inv()
      print("O_inv")
      display(O_inv)

      zerone= Matrix([[0 for i in range(A.shape[0]-1)]+[1]])
      zerone = zerone.transpose()
      print("zeone")
      display(zerone)
      print(" L = O_inv*Alpha_d(A)* [[0][.][0][1]] : = ")
      L = alpha_d*o_inv*zerone
      display(L)


In [ ]:
def Bass_gura_formaula(A,T,eigenvalues):
  print("bass_gura method to find observer gain control L")
  coefs = coefs_of_si_A_detrm(A)
  coefs = Matrix(coefs[:-1])
  diff = []
  Alphas=[]
  if eigenvalues:
      Alphas = coefs_of_si_A_detrm(diag(eigenvalues,unpack=True))
      
  else:
      print("assuming ur putting a3,a2,a1 from the desired characteristic eq: a3*s2 + a2*s + a1")
      for i in range(A.shape[0]+1):
        c = int(input(f" input a{i+1} :"))
        Alphas.append(c)
  Alphas = Matrix(Alphas[:-1])
  diff = Alphas -coefs
  print("[[a1 - A1][.][an-1 - An-1]]")
  display(diff)
  L = T*diff
  print("L = (MO)_inv * diff of coef matrix :=")
  display(L)
  return
  

In [ ]:
display(A,B,C)

array([[ 0,  1,  0,  0],
       [ 0,  0, -1,  0],
       [ 0,  0,  0,  1],
       [ 0,  0,  5,  0]])

array([[ 0],
       [ 1],
       [ 0],
       [-2]])

array([[1, 0, 0, 0]])

In [ ]:
#endsem observer
A = np.array([[0,1,0,0],[0,0,-1,0],[0,0,0,1],[0,0,5,0]])
B = np.array([[0],[1],[0],[-2]])
C = np.array([[1,0,0,0]])
s = system()
s.define_ABCD(A,B,C)
coef =   coefs_of_si_A_detrm(A)

print(coef)
  # coef of det(sI-A) = 0; => a4.s**3 - a3.s**2 - a2.s- a1 = 0  as (a3, a2, a1), manually
T,obs_mat_C = s.build_T_for_observibility(coef[:-1])

det = obs_mat_C.det()
print(f"Determinant of Observibility matrix: {det}")
eigenvalues=[-1,-2,-1+I,-1-I]
Ackkermans_formula(A,B,C,obs_mat_C,eigenvalues=eigenvalues,mode='observer')
#A_ccf,B_ccf,C_ccf = ABC_similarity_transform(A,B,C,T)
#c = Controller(A_ccf,B_ccf,C_ccf,T)
#k,k_bar = c.pole_assg_Similarity_transform([-1,-3+I,-3-I])
#tracking_gain_N(A,B,C,k,show_calc=True)

Determinant of SI-A


 4      2
s  - 5⋅s 

[0, 0, -5, 0, 1]
 T matrix for observability representation
M := 


⎡0   -5  0  1⎤
⎢            ⎥
⎢-5  0   1  0⎥
⎢            ⎥
⎢0   1   0  0⎥
⎢            ⎥
⎣1   0   0  0⎦

O = [[C][CA][CA2]]


...

⎡1  0  0   0 ⎤
⎢            ⎥
⎢0  1  0   0 ⎥
⎢            ⎥
⎢0  0  -1  0 ⎥
⎢            ⎥
⎣0  0  0   -1⎦

zeone


⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣1⎦

 L = O_inv*Alpha_d(A)* [[0][.][0][1]] : = 


⎡-5 ⎤
⎢   ⎥
⎢-15⎥
⎢   ⎥
⎢-35⎥
⎢   ⎥
⎣-79⎦

In [ ]:
#endsem controller
A = np.array([[0,1,0,0],[0,0,-1,0],[0,0,0,1],[0,0,5,0]])
B = np.array([[0],[1],[0],[-2]])
C = np.array([[1,0,0,0]])
s = system()
s.define_ABCD(A,B,C)
coef =   coefs_of_si_A_detrm(A)

print(coef)
  # coef of det(sI-A) = 0; => a4.s**3 - a3.s**2 - a2.s- a1 = 0  as (a3, a2, a1), manually
T,Control_mat_C = s.build_T_for_controllibilty(coef[:-1])

det = Control_mat_C.det()
print(f"Determinant of Controllibility matrix: {det}")
A_ccf,B_ccf,C_ccf = ABC_similarity_transform(A,B,C,T)
c = Controller(A_ccf,B_ccf,C_ccf,T)
k,k_bar = c.pole_assg_Similarity_transform([-1,-3+I,-3-I])
tracking_gain_N(A,B,C,k,show_calc=True)

Determinant of SI-A


 4      2
s  - 5⋅s 

[0, 0, -5, 0, 1]
 T matrix for controllibility representation
M := 
[[0 -5 0 1]
 [-5 0 1 0]
 [0 1 0 0]
 [1 0 0 0]]
C = [B AB A2B]
[[  0   1   0   2]
 [  1   0   2   0]
 [  0  -2   0 -10]
 [ -2   0 -10   0]]
T = C.M :=
[[-3 0 1 0]
 [0 -3 0 1]
 [0 0 -2 0]
 [0 0 0 -2]]
Determinant of Controllibility matrix: 36


[1  0  0  0]

T_inv :=


...

 3      2            
s  + 7⋅s  + 16⋅s + 10

required coef of det of (si - lambda) , where lambda are the new pole location
[1, 7, 16, 10]
det(SI-(A-BK))


                2       3    4      2
k₁ + k₂⋅s + k₃⋅s  + k₄⋅s  + s  - 5⋅s 

solve for k1,k2,k.... and enter


KeyboardInterrupt: ignored

In [ ]:
Ackkermans_formula(A,B,C,Control_mat_C,eigenvalues=[-1,-2,-1+I,-1-I],mode='controller')


 Using Ackkermans formula for finding control input K/L for controller 

Determinant of SI-A


 4      3       2           
s  + 5⋅s  + 10⋅s  + 10⋅s + 4

4*I :


⎡4  0  0  0⎤
⎢          ⎥
⎢0  4  0  0⎥
⎢          ⎥
⎢0  0  4  0⎥
⎢          ⎥
⎣0  0  0  4⎦

10*A**1


...

⎡0   1    0    2 ⎤
⎢                ⎥
⎢1   0    2    0 ⎥
⎢                ⎥
⎢0   -2   0   -10⎥
⎢                ⎥
⎣-2  0   -10   0 ⎦

Cont_inv :=


⎡ 0    5/3    0    1/3 ⎤
⎢                      ⎥
⎢5/3    0    1/3    0  ⎥
⎢                      ⎥
⎢ 0    -1/3   0    -1/6⎥
⎢                      ⎥
⎣-1/3   0    -1/6   0  ⎦

 K = [0 ... 0 1]*C_inv*Alpha_d(A)  : = 


[-4/3  -10/3  -49/6  -25/6]

In [ ]:
#eg observer
A = np.array([[0,1,0],[0,0,1],[-8,-5,-3]])
B = np.array([[0],[0],[4]])
C = np.array([[2,-4,0]])
s = system()
s.define_ABCD(A,B,C)
coef =   coefs_of_si_A_detrm(A)

print(coef)  # coef of det(sI-A) = 0; => a4.s**3 - a3.s**2 - a2.s- a1 = 0  as (a3, a2, a1), manually

T_obs,obsrv_matrix = s.build_T_for_observibility(coef[:-1])
eigenvalues = [-1+I,-1-I,-5]
A_obs,B_obs,C_obs = ABC_similarity_transform(A,B,C,T)
c = Controller(A_obs,B_obs,C_obs,T)
k,k_bar = c.pole_assg_Similarity_transform_2(eigenvalues)
Bass_gura_formaula(A,T_obs,eigenvalues)
Ackkermans_formula(A,B,C,obsrv_matrix,eigenvalues=eigenvalues,mode='observer')


Determinant of SI-A


 3      2          
s  + 3⋅s  + 5⋅s + 8

[8, 5, 3, 1]
 T matrix for observability representation
M := 


⎡5  3  1⎤
⎢       ⎥
⎢3  1  0⎥
⎢       ⎥
⎣1  0  0⎦

O = [[C][CA][CA2]]


...

⎡  27               ⎤
⎢ ───   1/13   2/91 ⎥
⎢ 182               ⎥
⎢                   ⎥
⎢-16                ⎥
⎢────   1/26   1/91 ⎥
⎢ 91                ⎥
⎢                   ⎥
⎣-8/91  -3/13  1/182⎦

zeone


⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣1⎦

 L = O_inv*Alpha_d(A)* [[0][.][0][1]] : = 


⎡1/7 ⎤
⎢    ⎥
⎢-13 ⎥
⎢────⎥
⎢ 14 ⎥
⎢    ⎥
⎢ 11 ⎥
⎢ ── ⎥
⎣ 14 ⎦

In [ ]:
A = np.array([[0,1,0],[0,-2,0.5],[0,0,-0.5]])
B = np.array([[0],[0.5],[0.5]])
C = np.array([[0,0,1]])
s = system()
s.define_ABCD(A,B,C)
coef =   coefs_of_si_A_detrm(A)

print(coef)
  # coef of det(sI-A) = 0; => a4.s**3 - a3.s**2 - a2.s- a1 = 0  as (a3, a2, a1), manually
T,Control_mat_C = s.build_T_for_controllibilty(coef[:-1])

det = Control_mat_C.det()
print(f"Determinant of Controllibility matrix: {det}")
A_ccf,B_ccf,C_ccf = ABC_similarity_transform(A,B,C,T)
c = Controller(A_ccf,B_ccf,C_ccf,T)
k,k_bar = c.pole_assg_Similarity_transform([-1,-3+I,-3-I])
tracking_gain_N(A,B,C,k,show_calc=True)

Determinant of SI-A


 3        2        
s  + 2.5⋅s  + 1.0⋅s

[0, 1.00000000000000, 2.50000000000000, 1]
 T matrix for controllibility representation
M := 
[[1.00000000000000 2.50000000000000 1]
 [2.50000000000000 1 0]
 [1 0 0]]
C = [B AB A2B]
[[ 0.     0.5   -0.75 ]
 [ 0.5   -0.75   1.375]
 [ 0.5   -0.25   0.125]]
T = C.M :=
[[0.500000000000000 0.500000000000000 0.0]
 [0 0.500000000000000 0.5]
 [0 1.00000000000000 0.5]]
Determinant of Controllibility matrix: 0.125000000000000


[0  0  1]

T_inv :=


...

(A-Bk)^-1 :=


⎡-2.6  -0.55  0.35⎤
⎢                 ⎥
⎢1.0     0     0  ⎥
⎢                 ⎥
⎣2.0    1.0   -1.0⎦

G_cl(0) = -C*(A-BK)^-1 *B := 


[0]

NonInvertibleMatrixError: ignored

In [ ]:
#eg 
A = np.array([[-5,-2],[2,0]])
B = np.array([[0.5],[0]])
C = np.array([[0,1]])
s = system()
s.define_ABCD(A,B,C)
coef =   coefs_of_si_A_detrm(A)

print(coef)
  # coef of det(sI-A) = 0; => a4.s**3 - a3.s**2 - a2.s- a1 = 0  as (a3, a2, a1), manually
T,Control_mat_C = s.build_T_for_controllibilty(coef[:-1])
det = Control_mat_C.det()
print(f"Determinant of Controllibility matrix: {det}")
A_bar,B_bar,C_bar = ABC_similarity_transform(A,B,C,T)
c = Controller(A,B,C,T)
k,k_bar = c.pole_assg_Similarity_transform([-4+2.75*I,-4-2.75*I])
tracking_gain_N(A,B,C,k,show_calc=True)

Determinant of SI-A


 2          
s  + 5⋅s + 4

[4, 5, 1]
 T matrix for controllibility representation
M := 
[[5 1]
 [1 0]]
C = [B AB A2B]
[[ 0.5 -2.5]
 [ 0.   1. ]]
T = C.M :=
[[0 0.5]
 [1.00000000000000 0.0]]


[0  1]

T_inv :=


...

⎡         0                  0.5        ⎤
⎢                                       ⎥
⎣-0.0851063829787234  -0.340425531914894⎦

G_cl(0) = -C*(A-BK)^-1 *B := 


[0.0425531914893617]

N_bar


[23.5]

In [ ]:
Ackkermans_formula(A,B,C,Control_mat_C,eigenvalues=[-4+2.75*I,-4-2.75*I],mode='controller')


 Using Ackkermans formula for finding control input K/L for controller 

Determinant of SI-A


 2                
s  + 8⋅s + 23.5625

23.5625000000000*I :


⎡23.5625     0   ⎤
⎢                ⎥
⎣   0     23.5625⎦

8*A**1


...

⎡0.5  -2.5⎤
⎢         ⎥
⎣0.0  1.0 ⎦

Cont_inv :=


⎡2.0  5.0⎤
⎢        ⎥
⎣ 0   1.0⎦

 K = [0 ... 0 1]*C_inv*Alpha_d(A)  : = 


[6.0  19.5625]

In [ ]:
A = np.array([[-1,1.5],[1,-2]])
B = np.array([[1],[0]])
C = np.array([[1,0]])
s = system()
s.define_ABCD(A,B,C)
coef =   coefs_of_si_A_detrm(A)

print(coef)  # coef of det(sI-A) = 0; => a4.s**3 - a3.s**2 - a2.s- a1 = 0  as (a3, a2, a1), manually

T_obs,obsrv_matrix = s.build_T_for_observibility(coef[1:])
Bass_gura_formaula(A,T_obs,[-4,-3])
Ackkermans_formula(A,B,C,obsrv_matrix,eigenvalues=[-4,-3],mode='observer')


Determinant of SI-Lambda


 2              
s  + 3.0⋅s + 0.5

[0.500000000000000, 3.00000000000000, 1]
 T matrix for observability representation
M := 


⎡1  1⎤
⎢    ⎥
⎣1  0⎦

O = [[C][CA][CA2]]


...

⎡       1.0                 0        ⎤
⎢                                    ⎥
⎣0.666666666666667  0.666666666666667⎦

zeone


⎡0⎤
⎢ ⎥
⎣1⎦

 L = O_inv*Alpha_d(A)* [[0][.][0][1]] : = 


⎡      6.0       ⎤
⎢                ⎥
⎣6.33333333333333⎦

In [ ]:

A = np.array([[0,1,1],[1,-1,0],[0,2,0]],dtype='int32')
B = np.array([[1],[0],[1]])
C = np.array([[1,0,0]])
s = system()
s.define_ABCD(A,B,C)
coef =   coefs_of_si_A_detrm(A)

print(coef[:-1])  # coef of det(sI-A) = 0; => a4.s**3 - a3.s**2 - a2.s- a1 = 0  as (a3, a2, a1), manually
T,Control_mat_C = s.build_T_for_controllibilty(coef[:-1])
A_bar,B_bar,C_bar = ABC_similarity_transform(A,B,C,T)
c = Controller(A_bar,B_bar,C_bar,T)
k = c.pole_assg_Similarity_transform([-1,-1,-1])
#tracking_gain_N(A,B,C,k,show_calc=True)

Determinant of SI-Lambda


 3    2        
s  + s  - s - 2

[-2, -1, 1]
 T matrix for controllibility representation
M := 
[[-1 1 1]
 [1 1 0]
 [1 0 0]]
C = [B AB A2B]
[[1 1 1]
 [0 1 0]
 [1 0 2]]
T = C.M :=
[[1 2 1]
 [1 1 0]
 [1 1 1]]


[1  0  0]

T_inv :=


...

⎡-1  0   0 ⎤
⎢          ⎥
⎢-1  -1  0 ⎥
⎢          ⎥
⎣0   -1  -1⎦

G_cl(0) = -C*(A-BK)^-1 *B := 


[1]

N_bar


[1]

In [ ]:
Ackkermans_formula_for_K(A,B,C,Control_mat_C,[-1,-1,-1])


 Using Ackkermans formula for finding control input k 

Determinant of SI-Lambda


 3      2          
s  + 3⋅s  + 3⋅s + 1

1*I :


⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

3*A**1


...

⎡1  1  1⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣1  0  2⎦

Cont_inv :=


⎡2   -2  -1⎤
⎢          ⎥
⎢0   1   0 ⎥
⎢          ⎥
⎣-1  1   1 ⎦

 K = [0 ... 0 1]*C_inv*Alpha_d(A)  : = 


[1  1  1]

In [ ]:
x= symbols('x')

In [ ]:
B = np.array([[0],[0],[1]])

In [ ]:
A = np.array([[1,0,0],[1,2,0],[1,0,-1]])

In [ ]:
s = eye(3)*x
sI = s - A
det = sI.det()
dicta = det.as_coefficients_dict()
det

x**3 - 3*x**2 - 4*x

In [ ]:
type(sI.rank())

int

In [ ]:
print(dicta)

defaultdict(<class 'int'>, {x**3: 1, x: -4, x**2: -3})


In [ ]:
T

Matrix([
[-1, 0, 0],
[ 0, 1, 0],
[ 0, 0, 2]])

In [ ]:
eigenvects = Matrix()

In [ ]:
A_s.inv()

In [ ]:
A_s = Matrix(A)
l = list(A_s)
l,A_s

([0, 0, 0, 1, 2, 0, 1, 0, -1], Matrix([
 [0, 0,  0],
 [1, 2,  0],
 [1, 0, -1]]))

In [ ]:
A_s.eigenvects()

[(-1, 1, [Matrix([
   [   0],
   [-1/2],
   [   1]])]), (0, 1, [Matrix([
   [0],
   [0],
   [1]])]), (4, 1, [Matrix([
   [4/5],
   [  0],
   [  1]])])]

In [ ]:
#if A is in CCF , finding diagonalized canonical form
A = np.array([[0,1],[3,2]])
diagonalize_from_CCF(A)

A in CCF
Matrix is singular
eigenvalues  := [-1.  3.]
Matrix has unique eigen values
calculation T from the eigen vectors and mapping [1 lmda**1 lmda**2 ...]
Given A to be represented = 
[[0 1]
 [3 2]]
controllibility matrix T = 
[[ 1.  1.]
 [-1.  3.]]
T inverse = 
[[ 0.75 -0.25]
 [ 0.25  0.25]]
 T_inv @ A = 
[[-0.75  0.25]
 [ 0.75  0.75]]
 representation_of_A = T_inv @ A @ T =  
[[-1.  0.]
 [-0.  3.]]


In [ ]:
#for controllability transformation

A = np.array([[0,1,1],[1,-1,0],[0,2,0]])
B = np.array([[1],[0],[1]])
C = np.array([[1,0,0]])
s = system()
s.define_ABCD(A,B,C)
coef = [2,-1,1]     # coef of det(sI-A) = 0; => s**3 - a1s**2 - a2s- a3 = 0  as (a3, a2, a1), manually
T = s.build_T_for_controllibilty(coef)
s.ABC_similarity_transform(T)

 T matrix for controllibility representation
M := 
[[-1  1  1]
 [ 1  1  0]
 [ 1  0  0]]
C = [B AB A2B]
[[1 1 1]
 [0 1 0]
 [1 0 2]]
T = C.M :=
[[1 2 1]
 [1 1 0]
 [1 1 1]]
Matrix is singular
Given A to be represented = 
[[ 0  1  1]
 [ 1 -1  0]
 [ 0  2  0]]
 T  := 
[[1 2 1]
 [1 1 0]
 [1 1 1]]
T inverse := 
[[-1.  1.  1.]
 [ 1. -0. -1.]
 [ 0. -1.  1.]]
 T_inv . A = 
[[ 1.  0. -1.]
 [ 0. -1.  1.]
 [-1.  3.  0.]]
 representation_of_A = T_inv . A . T =  
[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 2.  1. -1.]]
A_bar = T_inv . A . T 
[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 2.  1. -1.]]
B_bar = T_inv . B
[[0.]
 [0.]
 [1.]]
C_bar = C . T
[[1 2 1]]


In [ ]:
# for observability transformation
A = np.array([[1,2,1],[0,1,3],[1,1,1]])

B = np.array([[1],[0],[1]])
C = np.array([[1,1,0]])
s = system()
s.define_ABCD(A,B,C)
coef = [2,-1,-3]    # coef of det(sI-A) = s3 - a1s2 - a2s- a3 as (a3, a2, a1) manually
T = s.build_T_for_observibility([2,-1,-3])
s.ABC_similarity_transform(T)

 T matrix for observability representation
M := 
[[-1 -3  1]
 [-3  1  0]
 [ 1  0  0]]
O = [[C][CA][CA2]]
[[ 1  1  0]
 [ 1  3  4]
 [ 5  9 14]]
T = inv_(MO) :=
[[ 0.33333333 -0.16666667  0.33333333]
 [-0.33333333  0.16666667  0.66666667]
 [ 0.16666667  0.16666667  0.16666667]]
Matrix is singular
Given A to be represented = 
[[1 2 1]
 [0 1 3]
 [1 1 1]]
 T  := 
[[ 0.33333333 -0.16666667  0.33333333]
 [-0.33333333  0.16666667  0.66666667]
 [ 0.16666667  0.16666667  0.16666667]]
T inverse := 
[[ 1.00000000e+00 -1.00000000e+00  2.00000000e+00]
 [-2.00000000e+00  0.00000000e+00  4.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  3.33066907e-16]]
 T_inv . A = 
[[3. 3. 0.]
 [2. 0. 2.]
 [1. 3. 4.]]
 representation_of_A = T_inv . A . T =  
[[-0. -0.  3.]
 [ 1. -0.  1.]
 [-0.  1.  3.]]
A_bar = T_inv . A . T 
[[-0. -0.  3.]
 [ 1. -0.  1.]
 [-0.  1.  3.]]
B_bar = T_inv . B
[[3.]
 [2.]
 [1.]]
C_bar = C . T
[[-5.55111512e-17 -5.55111512e-17  1.00000000e+00]]


In [ ]:

print(A)

[[  0   1   0]
 [  0   0   1]
 [ -6 -11  -6]]


In [ ]:



A = np.array([[2,1],[0,3]])
T = np.array([[1,3],[5,9]]).T
B = np.array([[2],[3]])
C = np.array([[1,0]])
s = system()
s.define_ABCD(A,B,C)
s.ABC_similarity_transform(T)


Matrix is singular
Given A to be represented = 
[[2 1]
 [0 3]]
controllibility matrix T = 
[[1 5]
 [3 9]]
T inverse = 
[[-1.5         0.83333333]
 [ 0.5        -0.16666667]]
 T_inv A = 
[[-3.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  2.77555756e-17]]
 representation_of_A = T_inv * A * T =  
[[ 0 -6]
 [ 1  5]]
A_bar = T_inv . A . T 
[[ 0 -6]
 [ 1  5]]
B_bar = T_inv . B
[[-0.5]
 [ 0.5]]
C_bar = C . T
[[1 5]]


In [ ]:
s.A = A

In [ ]:
A = np.array([[-4,3,2],[2,-6,1],[1,2,-8]])
B = np.array([[1],[2],[3]])
C = np.array([[1,0,0]])
s = system()
s.define_ABCD(A,B,C)
coef = [113,94,18]     # coef of det(sI-A) = 0; => s**3 - a1s**2 - a2s- a3 = 0  as (a3, a2, a1), manually
T = s.build_T_for_controllibilty(coef)
s.ABC_similarity_transform(T)

 T matrix for controllibility representation
M := 
[[94 18  1]
 [18  1  0]
 [ 1  0  0]]
C = [B AB A2B]
[[  1   8 -91]
 [  2  -7  39]
 [  3 -19 146]]
T = C.M :=
[[147  26   1]
 [101  29   2]
 [ 86  35   3]]
Matrix is non singular
Given A to be represented = 
[[-4  3  2]
 [ 2 -6  1]
 [ 1  2 -8]]
 T  := 
[[147  26   1]
 [101  29   2]
 [ 86  35   3]]
T inverse := 
[[  0.12686567  -0.32089552   0.17164179]
 [ -0.97761194   2.64925373  -1.44029851]
 [  7.76865672 -21.70895522  12.21641791]]
 T_inv . A = 
[[  -0.97761194    2.64925373   -1.44029851]
 [   7.76865672  -21.70895522   12.21641791]
 [ -62.2761194   177.99253731 -103.90298507]]
 representation_of_A = T_inv . A . T =  
[[   0.    1.    0.]
 [  -0.   -0.    1.]
 [-113.  -94.  -18.]]
A_bar = T_inv . A . T 
[[   0.    1.    0.]
 [  -0.   -0.    1.]
 [-113.  -94.  -18.]]
B_bar = T_inv . B
[[-1.94289029e-16]
 [ 8.88178420e-16]
 [ 1.00000000e+00]]
C_bar = C . T
[[147  26   1]]
